In [112]:
import os
import re

from configparser import ConfigParser, NoOptionError
from sqlalchemy import create_engine
import numpy as np
import pandas as pd

from bioblend.galaxy import GalaxyInstance
# from bioblend.galaxy.histories import HistoryClient
# from bioblend.galaxy.workflows import WorkflowClient
from bioblend.galaxy.tools import ToolClient

In [18]:
galaxy_url = 'http://127.0.0.1:8080'
api_key = os.getenv('GALAXY_API_KEY')
gi = GalaxyInstance(url=galaxy_url, key=api_key)

### Connect to the database

In [3]:
galaxy_path = os.getenv('GALAXY_DIRECTORY')
config_file = os.path.join(galaxy_path, 'config/galaxy.ini.sample')
db_path = ''

if os.path.isfile(config_file):
    config = ConfigParser()
    config.read_file(open(config_file))
    
    try:
        # Get database path     
        db_path = config.get('app:main', 'database_connection')
#         db_path = 'postgresql://galaxy@localhost:5432/galaxy'  # temporary
            
    except NoOptionError:
        # Default
        db_path = 'sqlite:///' + os.path.join(galaxy_path, 'database/universe.sqlite')

engine = create_engine(db_path)

In [ ]:
# wc = WorkflowClient(gi)
# wc.get_workflows()

In [ ]:
# hc = HistoryClient(gi)
# hc.get_histories()

## Connections

In [204]:
query_get_connections = '''
SELECT DISTINCT ws.id, ws.workflow_id, ws.tool_id,
    ws.tool_version, ws.label, wsa.annotation,
    wsc.output_step_id AS in_port, wsc.input_step_id AS out_port

FROM workflow_step as ws, stored_workflow as sw

LEFT JOIN workflow_step_connection AS wsc
    ON ws.id = wsc.output_step_id

LEFT JOIN workflow_step_annotation_association as wsa
    ON ws.id = wsa.workflow_step_id

WHERE ws.workflow_id = sw.latest_workflow_id AND ws.type != 'data_input'
'''
connections = pd.read_sql(query_get_connections, con=engine)
connections[['in_port', 'out_port']] = connections[['in_port', 'out_port']].fillna(0.0).applymap(np.int64)

# def get_tool_ids(port):
#     for port_id in connections[port]:
#         tool_id = connections.tool_id[connections.id == port_id]
#         yield tool_id.values[0] if not tool_id.empty else None

# in_port_ids = pd.Series(list(get_tool_ids('in_port')), name='in_port_tool')
# out_port_ids = pd.Series(list(get_tool_ids('out_port')), name='out_port_tool')
# connections = pd.concat([connections, in_port_ids, out_port_ids], axis=1)
# connections = connections[(connections.in_port != 0) & (connections.out_port != 0)]

connections

,id,workflow_id,tool_id,tool_version,label,annotation,in_port,out_port
0,18,6,Cut1,1.0.2,cut 1,cut 1 annot,18,20
1,19,6,Cut1,1.0.2,cut 2,cut 2 annot,19,20
2,20,6,cat1,1.0.0,output,output annot,0,0


## Tool information

In [201]:
query_get_tools = '''
SELECT DISTINCT ws.tool_id AS id, ws.tool_version AS version
FROM workflow_step as ws, stored_workflow as sw
WHERE ws.tool_id IS NOT NULL AND ws.workflow_id = sw.latest_workflow_id
'''
workflow_tools = pd.read_sql(query_get_tools, con=engine)
workflow_tools['flag'] = 1

tc = ToolClient(gi)
all_tools = pd.DataFrame.from_dict(tc.get_tools())

tools = pd.merge(all_tools, workflow_tools, on=['id', 'version'], how='left')
tools = tools[pd.notnull(tools['flag'])].drop(['flag'], axis=1)

tools

,description,edam_operations,edam_topics,form_style,id,labels,link,min_width,model_class,name,panel_section_id,panel_section_name,target,tool_shed_repository,version
98,columns from a table,[],[],regular,Cut1,[],/tool_runner?tool_id=Cut1,-1,Tool,Cut,textutil,Text Manipulation,galaxy_main,NaN,1.0.2
118,tail-to-head,[],[],regular,cat1,[],/tool_runner?tool_id=cat1,-1,Tool,Concatenate datasets,textutil,Text Manipulation,galaxy_main,NaN,1.0.0


In [189]:
tc.get_tools(name='KOBAS Annotate')

[{'description': 'KEGG Orthology Based Annotation System',
  'edam_operations': [],
  'edam_topics': [],
  'form_style': 'regular',
  'id': 'toolshed.g2.bx.psu.edu/repos/iuc/kobas/kobas_annotate/0.4',
  'labels': [],
  'link': '/tool_runner?tool_id=toolshed.g2.bx.psu.edu%2Frepos%2Fiuc%2Fkobas%2Fkobas_annotate%2F0.4',
  'min_width': -1,
  'model_class': 'Tool',
  'name': 'KOBAS Annotate',
  'panel_section_id': 'toolshed',
  'panel_section_name': 'ToolShed',
  'target': 'galaxy_main',
  'tool_shed_repository': {'changeset_revision': '83b998fa34ea',
   'name': 'kobas',
   'owner': 'iuc',
   'tool_shed': 'toolshed.g2.bx.psu.edu'},
  'version': '0.4'}]